In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import re
import cv2
import numpy as np
from torchvision import transforms
import random

# ensure determinism
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 147 * 147, 256)  # Adjusted input size
        self.fc2 = nn.Linear(256, 2) # predicting two variables (alfa x and betax)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class CustomDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = sorted(os.listdir(image_dir))
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)\.png"
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_name = self.image_files[index]
        image_path = os.path.join(self.image_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transform(image)
        matches = re.search(self.pattern, image_name)
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        alfa_x = variables[1] # alfax value
        beta_x = variables[2] # betax value
        return image, torch.tensor([alfa_x, beta_x]) # returns alfa x and betax as the labels

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 4
learning_rate = 0.01
num_epochs = 100

model = CNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

image_dir = "train"
dataset = CustomDataset(image_dir)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

total_steps = len(dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.4f}")

        # if (i + 1) % 10 == 0:
        #     print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.4f}")
    
filename = f"cnn_model3_bs{batch_size}_lr{learning_rate}_e{num_epochs}.pth"
torch.save(model.state_dict(), filename)


Epoch [1/100], Step [1/124], Loss: 50313.2656
Epoch [1/100], Step [2/124], Loss: 83558176.0000
Epoch [1/100], Step [3/124], Loss: 1444563.7500
Epoch [1/100], Step [4/124], Loss: 194330.6875
Epoch [1/100], Step [5/124], Loss: 163572.3906
Epoch [1/100], Step [6/124], Loss: 121341.5547
Epoch [1/100], Step [7/124], Loss: 87324.1016
Epoch [1/100], Step [8/124], Loss: 31212.8945
Epoch [1/100], Step [9/124], Loss: 28712.8086
Epoch [1/100], Step [10/124], Loss: 20108.7031
Epoch [1/100], Step [11/124], Loss: 11445.8398
Epoch [1/100], Step [12/124], Loss: 21383.6641
Epoch [1/100], Step [13/124], Loss: 7032.3975
Epoch [1/100], Step [14/124], Loss: 6054.6689
Epoch [1/100], Step [15/124], Loss: 2786.7783
Epoch [1/100], Step [16/124], Loss: 17885.3203
Epoch [1/100], Step [17/124], Loss: 9818.7402
Epoch [1/100], Step [18/124], Loss: 6827.3540
Epoch [1/100], Step [19/124], Loss: 8996.1719
Epoch [1/100], Step [20/124], Loss: 9756.1250
Epoch [1/100], Step [21/124], Loss: 4848.3760
Epoch [1/100], Step [2

In [4]:
# Model evaluation function
def evaluate_model(model, test_dataset):
    model.eval()
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    total_mae = torch.zeros(2, device=device)
    total_mape = torch.zeros(2, device=device)
    total_smape = torch.zeros(2, device=device)
    total_mse = torch.zeros(2, device=device)
    total_count = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            absolute_error = torch.abs(outputs - labels)
            total_mae += absolute_error.sum(dim=0)

            non_zero_mask = torch.abs(labels) > 1e-8
            percentage_error = (absolute_error / torch.abs(labels)) * 100
            total_mape += (percentage_error * non_zero_mask).sum(dim=0)

            smape = 200.0 * torch.abs(outputs - labels) / (torch.abs(outputs) + torch.abs(labels) + torch.finfo(torch.float32).eps)
            total_smape += smape.sum(dim=0)
            
            mse = (outputs - labels) ** 2
            total_mse += mse.sum(dim=0)

            total_count += labels.size(0)

    mae = total_mae / total_count
    mape = total_mape / total_count
    smape = total_smape / total_count
    rmse = torch.sqrt(total_mse / total_count)

    return mae.cpu().numpy(), mape.cpu().numpy(), smape.cpu().numpy(), rmse.cpu().numpy()

# # Include this in the training loop, after testing each model
# mae, mape, smape, rmse = evaluate_model(model, test_dataset)

# for i in range(2):
#     print(f"Metrics for prediction {i+1}:")
#     print(f"Mean Absolute Error: {mae[i]:.4f}")
#     print(f"Mean Absolute Percentage Error: {mape[i]:.2f}%")
#     print(f"Symmetric Mean Absolute Percentage Error: {smape[i]:.2f}%")
#     print(f"Root Mean Square Error: {rmse[i]:.4f}")
#     print()

In [7]:
# Import required libraries
import glob
import torch

# Load models and evaluate their performance
model_dir = './'  # Specify ysour directory where models are saved
test_dataset = CustomDataset('test')  # Specify your test dataset

model_performance = []

# Load each model and evaluate it
for model_file in glob.glob(model_dir + '/*.pth'):
    # Load model
    model = CNN()
    model.load_state_dict(torch.load(model_file))
    model.to(device)

    # Evaluate model
    mae, mape, smape, rmse = evaluate_model(model, test_dataset)
    aggregate_score = np.mean([mae, mape, smape, rmse])
    model_performance.append((model_file, mae, mape, smape, rmse, aggregate_score))

# Sort models based on aggregate score
model_performance.sort(key=lambda x: x[-1])

# Print the performance of each model
for model_info in model_performance:
    model_file, mae, mape, smape, rmse, aggregate_score = model_info
    print(f"Model: {model_file}, Aggregate Score: {aggregate_score:.4f}")
    print(f"Mean Absolute Error: {mae}")
    print(f"Mean Absolute Percentage Error: {mape}")
    print(f"Symmetric Mean Absolute Percentage Error: {smape}")
    print(f"Root Mean Square Error: {rmse}")
    print()


Model: .\current_best_model_b4_lr0.001_e500.pth, Aggregate Score: 38.7046
Mean Absolute Error: [ 0.9920414 50.312073 ]
Mean Absolute Percentage Error: [73.765594 27.613575]
Symmetric Mean Absolute Percentage Error: [62.775276 23.511852]
Root Mean Square Error: [ 1.2363389 69.42998  ]

Model: .\model-best-retest-shorter.pth, Aggregate Score: 64.4004
Mean Absolute Error: [ 2.4229126 60.97186  ]
Mean Absolute Percentage Error: [214.41547  26.38164]
Symmetric Mean Absolute Percentage Error: [99.80996 25.17152]
Root Mean Square Error: [ 2.9683042 83.06172  ]

